___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Latent Dirichlet Allocation

## Data

We will be using articles from NPR (National Public Radio), obtained from their website [www.esv.org](http://www.esv.org)

In [1]:
import pandas as pd

In [2]:
esv = pd.read_csv('Bible.csv')

In [3]:
esv.head()

,Book,Content
0,Genesis,"In the beginning, God created the heavens and ..."
1,Exodus,These are the names of the sons of Israel who ...
2,Leviticus,The LORD called Moses and spoke to him from th...
3,Numbers,The LORD spoke to Moses in the wilderness of S...
4,Deuteronomy,These are the words that Moses spoke to all Is...


In [4]:
esv

,Book,Content
0,Genesis,"In the beginning, God created the heavens and ..."
1,Exodus,These are the names of the sons of Israel who ...
2,Leviticus,The LORD called Moses and spoke to him from th...
3,Numbers,The LORD spoke to Moses in the wilderness of S...
4,Deuteronomy,These are the words that Moses spoke to all Is...
...,...,...
61,1 John,"That which was from the beginning, which we ha..."
62,2 John,"The elder to the elect lady and her children, ..."
63,3 John,"The elder to the beloved Gaius, whom I love in..."
64,Jude,"Jude, a servant of Jesus Christ and brother of..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [6]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [7]:
dtm = cv.fit_transform(esv['Content'])

In [8]:
dtm

<66x7619 sparse matrix of type '<class 'numpy.int64'>'
	with 58977 stored elements in Compressed Sparse Row format>

## LDA

In [9]:
from sklearn.decomposition import LatentDirichletAllocation

In [40]:
LDA = LatentDirichletAllocation(n_components=12)

In [41]:
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

LatentDirichletAllocation(n_components=12)

## Showing Stored Words

In [42]:
len(cv.get_feature_names())

7619

In [43]:
import random

In [44]:
#Dempnstation of random selection of words
for i in range(9):
    random_word_id = random.randint(0,8)
    print(cv.get_feature_names()[random_word_id])

100
100
100
112
110
112
120
110
000


### Showing Top Words Per Topic

In [45]:
len(LDA.components_)

12

In [46]:
LDA.components_

array([[8.33334993e-02, 8.33333333e-02, 8.33333333e-02, ...,
        8.33333333e-02, 8.33333333e-02, 8.33333333e-02],
       [1.15454769e+01, 8.33333333e-02, 8.33333333e-02, ...,
        8.33337121e-02, 8.33335424e-02, 2.43128718e+00],
       [1.41080071e+01, 8.33333333e-02, 8.33333333e-02, ...,
        8.33333333e-02, 8.33333333e-02, 8.33333333e-02],
       ...,
       [1.61701163e+02, 2.08333292e+00, 2.08333292e+00, ...,
        1.00833289e+01, 8.33349109e-02, 6.73537732e+00],
       [8.33333333e-02, 8.33333333e-02, 8.33333333e-02, ...,
        8.33333333e-02, 8.33333333e-02, 8.33333333e-02],
       [1.44027626e-01, 8.33336842e-02, 8.33336842e-02, ...,
        8.33333691e-02, 8.33336248e-02, 8.33337259e-02]])

In [47]:
len(LDA.components_)


12

In [48]:
type(LDA.components_)

numpy.ndarray

In [49]:
#7 topics x 610 words
LDA.components_.shape

(12, 7619)

In [50]:
single_topic = LDA.components_[0]

In [51]:
# Returns the indices of this sorted array from greatest valeus to least.
single_topic.argsort()

array([3621, 5367, 1237, ..., 2468, 3710, 1259], dtype=int64)

In [52]:
#ARGSORT ---> returns INDEX POSITIONS SORTED FROM LEAST --> GREATEST
#Looking ror TOP 10 CALUES (10 GREATEST VALUES)
#meaning I want the LAST 10 VALUES of ARGSORT()
#it is saying GRAB the last 10 values of .argsort() (highest prob words for this topic) 
# Word most representative of this topic
single_topic[-9]

0.0833333333333521

In [53]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([2974, 4110, 6820, 1031, 3863, 6387, 3928, 2468, 3710, 1259],
      dtype=int64)

In [54]:
top_word_indices = single_topic.argsort()[-10:]

In [55]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

grace
love
things
brothers
know
spirit
law
faith
jesus
christ


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [56]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 12 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

THE TOP 12 WORDS FOR TOPIC #0
['grace', 'love', 'things', 'brothers', 'know', 'spirit', 'law', 'faith', 'jesus', 'christ']


THE TOP 12 WORDS FOR TOPIC #1
['aaron', 'day', 'offering', 'land', 'said', 'moses', 'israel', 'people', 'lord', 'shall']


THE TOP 12 WORDS FOR TOPIC #2
['make', 'know', 'israel', 'behold', 'nations', 'says', 'land', 'declares', 'lord', 'shall']


THE TOP 12 WORDS FOR TOPIC #3
['grace', 'boast', 'spirit', 'things', 'did', 'glory', 'faith', 'jesus', 'lord', 'christ']


THE TOP 12 WORDS FOR TOPIC #4
['father', 'son', 'came', 'sons', 'men', 'went', 'saul', 'lord', 'david', 'said']


THE TOP 12 WORDS FOR TOPIC #5
['heart', 'steadfast', 'shall', 'people', 'forever', 'praise', 'like', 'love', 'let', 'lord']


THE TOP 12 WORDS FOR TOPIC #6
['deceivers', 'storehouse', 'autumn', 'fare', 'deceiver', 'amittai', 'ink', 'demetrius', 'vomited', 'nonsense']


THE TOP 12 WORDS FOR TOPIC #7
['people', 'make', 'altar', 'land', 'blood', 'unclean', 'priest', 'offering', 'lord', 'sha

### Attaching Discovered Topic Labels to Original Articles

In [57]:
dtm

<66x7619 sparse matrix of type '<class 'numpy.int64'>'
	with 58977 stored elements in Compressed Sparse Row format>

In [58]:
dtm.shape

(66, 7619)

In [59]:
len(esv)

66

In [60]:
topic_results = LDA.transform(dtm)

In [61]:
topic_results.shape

(66, 12)

In [62]:
topic_results[0]

array([8.07126209e-06, 8.07141605e-06, 8.07136299e-06, 8.07121639e-06,
       9.99911216e-01, 8.07131229e-06, 8.07102503e-06, 8.07135205e-06,
       8.07135541e-06, 8.07130801e-06, 8.07112976e-06, 8.07130511e-06])

In [63]:
topic_results[0].round(2)

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [65]:
topic_results[0].argmax()

4

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [66]:
esv.head()

,Book,Content,Topic
0,Genesis,"In the beginning, God created the heavens and ...",22
1,Exodus,These are the names of the sons of Israel who ...,15
2,Leviticus,The LORD called Moses and spoke to him from th...,0
3,Numbers,The LORD spoke to Moses in the wilderness of S...,15
4,Deuteronomy,These are the words that Moses spoke to all Is...,19


In [67]:
topic_results.argmax(axis=1)

array([ 4,  1,  7,  1,  1,  9,  9,  4,  4,  4,  9,  9,  9,  9,  9,  9,  8,
        8,  5,  8,  8,  8,  8,  9,  8,  2,  8,  8,  8,  8,  8, 11,  8,  8,
        8, 10,  9,  8,  8, 11, 11, 11, 11, 11,  0,  5,  3,  0,  0,  0,  0,
        5,  0,  5,  0,  5,  0,  3,  5,  5,  3,  0,  0,  0,  5,  8],
      dtype=int64)

In [68]:
esv['Topic'] = topic_results.argmax(axis=1)

In [69]:
esv.head(60)

,Book,Content,Topic
0,Genesis,"In the beginning, God created the heavens and ...",4
1,Exodus,These are the names of the sons of Israel who ...,1
2,Leviticus,The LORD called Moses and spoke to him from th...,7
3,Numbers,The LORD spoke to Moses in the wilderness of S...,1
4,Deuteronomy,These are the words that Moses spoke to all Is...,1
5,Joshua,After the death of Moses the servant of the LO...,9
6,Judges,"After the death of Joshua, the people of Israe...",9
7,Ruth,In the days when the judges ruled there was a ...,4
8,1 Samuel,There was a certain man of Ramathaim-zophim of...,4
9,2 Samuel,"After the death of Saul, when David had return...",4


## Great work!

In [70]:
import watermark
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p pandas,numpy,sklearn,watermark 

#%watermark -d -n -t -i -z -u -v -h -m -g -w


# date
print (" ")
%watermark -u -n -t -z

ModuleNotFoundError: No module named 'watermark'